In [ ]:
%load_ext line_profiler

In [2]:
from sklearn.datasets import make_blobs

from divik.cluster import KMeans, DunnDiviK
import divik.cluster._kmeans._core
import divik.cluster._kmeans._gap
import divik.cluster._kmeans._initialization
import divik.cluster._divik._dunn
import divik.score._sampled_gap
import divik.score._gap

In [3]:
X, y = make_blobs(n_samples=100_000, n_features=100, centers=10, random_state=42)

In [4]:
kmeans = KMeans(n_clusters=10, init='kdtree')

In [5]:
mdl = DunnDiviK(distance='euclidean', filter_type='auto', n_jobs=1, features_percentage=0.2, verbose=True)

# Generic timing

In [6]:
%lprun \
    -f divik.cluster._kmeans._initialization.KDTreeInitialization.__call__ \
    -f divik.cluster._kmeans._core._KMeans.__call__ \
    kmeans.fit(X)

Timer unit: 1e-06 s

Total time: 0.755221 s
File: /app/divik/cluster/_kmeans/_core.py
Function: __call__ at line 113

Line #      Hits         Time  Per Hit   % Time  Line Contents
   113                                               def __call__(self, data: Data, number_of_clusters: int) \
   114                                                       -> Tuple[IntLabels, Centroids]:
   115         1         19.0     19.0      0.0          _validate_kmeans_input(data, number_of_clusters)
   116         1          8.0      8.0      0.0          if number_of_clusters == 1:
   117                                                       return np.zeros((data.shape[0], 1), dtype=int), \
   118                                                              np.mean(data, axis=0, keepdims=True)
   119         1         19.0     19.0      0.0          data = data.reshape(data.shape, order='C')
   120         1          9.0      9.0      0.0          if self.normalize_rows:
   121                     

# Init timing

In [7]:
%lprun \
    -f divik.cluster._kmeans._initialization.KDTreeInitialization.__call__ \
    -f divik.cluster._kmeans._initialization._find_residuals \
    -f divik.cluster._kmeans._initialization.make_tree \
    kmeans.fit(X)

Timer unit: 1e-06 s

Total time: 0.003831 s
File: /app/divik/cluster/_kmeans/_initialization.py
Function: _find_residuals at line 27

Line #      Hits         Time  Per Hit   % Time  Line Contents
    27                                           def _find_residuals(data: Data, sample_weight=None) -> np.ndarray:
    28         1          9.0      9.0      0.2      features = data.T
    29         1         10.0     10.0      0.3      assumed_ys = features[0]
    30         1         33.0     33.0      0.9      modelled_xs = np.hstack([np.ones((data.shape[0], 1)),
    31         1         55.0     55.0      1.4                              features[1:].T])
    32         1         19.0     19.0      0.5      lr = LinearRegression().fit(modelled_xs, assumed_ys,
    33         1       3474.0   3474.0     90.7                                  sample_weight=sample_weight)
    34         1        223.0    223.0      5.8      residuals = np.abs(lr.predict(modelled_xs) - assumed_ys)
    35     

In [8]:
%lprun \
    -f divik.cluster._kmeans._initialization.KDTreeInitialization.__call__ \
    -f divik.cluster._kmeans._initialization._find_residuals \
    -f divik.cluster._kmeans._initialization.make_tree \
    mdl.fit(X)

100%|██████████| 2/2 [00:21<00:00, 11.62s/it]
                                             
100%|██████████| 9/9 [00:02<00:00,  4.05it/s]
                                             
100%|██████████| 100000/100000 [02:02<00:00, 814.16it/s]


Timer unit: 1e-06 s

Total time: 0.396158 s
File: /app/divik/cluster/_kmeans/_initialization.py
Function: _find_residuals at line 27

Line #      Hits         Time  Per Hit   % Time  Line Contents
    27                                           
    28       140       1716.0     12.3      0.4  def _find_residuals(data: Data, sample_weight=None) -> np.ndarray:
    29       140       1479.0     10.6      0.4      features = data.T
    30       140       4683.0     33.5      1.2      assumed_ys = features[0]
    31       140       7533.0     53.8      1.9      modelled_xs = np.hstack([np.ones((data.shape[0], 1)),
    32       140       2914.0     20.8      0.7                              features[1:].T])
    33       140     338280.0   2416.3     85.4      lr = LinearRegression().fit(modelled_xs, assumed_ys,
    34       140      38303.0    273.6      9.7                                  sample_weight=sample_weight)
    35       140       1250.0      8.9      0.3      residuals = np.abs